# Lab 10 — Neural Networks: PyTorch 입문

> **강의 시간:** 약 2시간  
> **주제:** 퍼셉트론부터 다층 신경망(MLP) 학습까지

---

## 학습 목표

| # | 목표 | 예상 시간 |
|---|---|---|
| 1 | 퍼셉트론 원리와 NumPy 구현 | 25분 |
| 2 | 활성화 함수 (ReLU, Sigmoid, Tanh) 특성 분석 | 15분 |
| 3 | 순전파(Forward Pass) 직접 계산 | 20분 |
| 4 | PyTorch 기초: Tensor, Autograd, nn.Module | 25분 |
| 5 | MLP 학습 루프 및 성능 분석 | 30분 |

---

**데이터셋:**
- 시각화용: 합성 2D 데이터 (make_moons)
- 분류: Breast Cancer Wisconsin (sklearn) — 30개 특성, 이진 분류

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

from sklearn.datasets import load_breast_cancer, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 한글 폰트
_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

rng = np.random.default_rng(42)
torch.manual_seed(42)
np.random.seed(42)

print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available : {torch.cuda.is_available()}')
device = torch.device('cpu')
print(f'Using device   : {device}')

---
## Part 1. 퍼셉트론 (Perceptron)

### 1-1. 퍼셉트론이란?

퍼셉트론(Perceptron)은 인공 신경망의 가장 기본 단위입니다.

```
입력 (x₁, x₂, ..., xₙ)
  × 가중치 (w₁, w₂, ..., wₙ)
       ↓
  가중합: z = Σ(wᵢ · xᵢ) + b
       ↓
  활성화 함수: ŷ = f(z)
       ↓
  출력 (ŷ)
```

| 구성 요소 | 역할 |
|---|---|
| **입력 (x)** | 특성 벡터 |
| **가중치 (w)** | 각 입력의 중요도 |
| **편향 (b)** | 결정 경계 이동 |
| **활성화 함수 (f)** | 비선형성 추가 |

**학습 규칙:** 예측 오류에 비례하여 가중치를 조정
$$w \leftarrow w + \eta \cdot (y - \hat{y}) \cdot x$$

In [ ]:
# NumPy로 구현한 퍼셉트론
class Perceptron:
    """NumPy로 구현한 단층 퍼셉트론"""

    def __init__(self, n_features, lr=0.1, n_epochs=50):
        self.w = np.zeros(n_features)
        self.b = 0.0
        self.lr = lr
        self.n_epochs = n_epochs
        self.errors_ = []

    def predict(self, X):
        z = X @ self.w + self.b
        return np.where(z >= 0, 1, 0)

    def fit(self, X, y):
        self.errors_ = []
        for epoch in range(self.n_epochs):
            errors = 0
            for xi, yi in zip(X, y):
                y_hat = self.predict(xi.reshape(1, -1))[0]
                update = self.lr * (yi - y_hat)
                self.w += update * xi
                self.b += update
                errors += int(update != 0)
            self.errors_.append(errors)
        return self


# AND 게이트 학습
X_and = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=float)
y_and = np.array([0, 0, 0, 1])

pct = Perceptron(n_features=2, lr=0.1, n_epochs=20)
pct.fit(X_and, y_and)

print('=== AND 게이트 학습 결과 ===')
print(f'{"입력 (x1, x2)":>20} | {"정답":>5} | {"예측":>5}')
print('-' * 38)
for xi, yi in zip(X_and, y_and):
    y_hat = pct.predict(xi.reshape(1, -1))[0]
    status = '✓' if yi == y_hat else '✗'
    print(f'{str(xi.astype(int)):>20} | {yi:>5} | {y_hat:>5} {status}')
print(f'\n학습된 가중치: w={pct.w.round(3)}, b={pct.b:.3f}')

# 학습 과정 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(range(1, len(pct.errors_) + 1), pct.errors_, 'o-', color='steelblue', lw=2)
axes[0].set_title('에포크별 오분류 수 (AND 게이트)')
axes[0].set_xlabel('에포크')
axes[0].set_ylabel('오분류 수')

x1_range = np.linspace(-0.5, 1.5, 100)
if abs(pct.w[1]) > 1e-6:
    boundary = -(pct.w[0] * x1_range + pct.b) / pct.w[1]
    axes[1].plot(x1_range, boundary, 'g--', lw=2.5, label='결정 경계')
axes[1].scatter(X_and[y_and == 0, 0], X_and[y_and == 0, 1],
                s=200, c='steelblue', label='0 (False)', zorder=5, marker='s')
axes[1].scatter(X_and[y_and == 1, 0], X_and[y_and == 1, 1],
                s=200, c='tomato', label='1 (True)', zorder=5, marker='^')
for xi, yi in zip(X_and, y_and):
    axes[1].annotate(f'({int(xi[0])},{int(xi[1])})', xy=(xi[0] + 0.05, xi[1] + 0.05), fontsize=11)
axes[1].set_title('AND 게이트 — 학습된 결정 경계')
axes[1].set_xlabel('x₁'); axes[1].set_ylabel('x₂')
axes[1].legend()
axes[1].set_xlim(-0.5, 1.5); axes[1].set_ylim(-0.5, 1.5)

plt.tight_layout()
plt.show()

### 1-2. 퍼셉트론의 한계 — XOR 문제

단일 퍼셉트론은 **선형으로 분리되지 않는** 문제를 풀 수 없습니다.

| 입력 | AND | OR | **XOR** |
|---|---|---|---|
| (0, 0) | 0 | 0 | **0** |
| (0, 1) | 0 | 1 | **1** |
| (1, 0) | 0 | 1 | **1** |
| (1, 1) | 1 | 1 | **0** |

XOR은 하나의 직선으로 분리할 수 없습니다. → **다층 퍼셉트론(MLP)** 이 필요합니다!

In [ ]:
# XOR에서 퍼셉트론 실패 시각화
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=float)
y_xor = np.array([0, 1, 1, 0])

pct_xor = Perceptron(n_features=2, lr=0.1, n_epochs=100)
pct_xor.fit(X_xor, y_xor)

print('=== XOR 게이트 퍼셉트론 결과 ===')
for xi, yi in zip(X_xor, y_xor):
    y_hat = pct_xor.predict(xi.reshape(1, -1))[0]
    status = '✓' if yi == y_hat else '✗'
    print(f'  입력 {xi.astype(int)} → 정답: {yi}, 예측: {y_hat} {status}')
print(f'\n100 에포크 후 마지막 에포크 오분류: {pct_xor.errors_[-1]}개  → 수렴 불가!')

# AND vs XOR 결정 경계 비교
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
x1_range = np.linspace(-0.5, 1.5, 100)

# AND - 선형 분리 가능
for xi, yi in zip(X_and, y_and):
    axes[0].scatter(xi[0], xi[1], c='tomato' if yi == 1 else 'steelblue',
                    s=300, zorder=5, marker='^' if yi == 1 else 's', edgecolors='black', lw=1.5)
    axes[0].annotate(f'({int(xi[0])},{int(xi[1])})={yi}', xy=(xi[0] + 0.07, xi[1] + 0.07), fontsize=11)
if abs(pct.w[1]) > 1e-6:
    boundary = -(pct.w[0] * x1_range + pct.b) / pct.w[1]
    axes[0].plot(x1_range, boundary, 'g-', lw=2.5, label='결정 경계')
axes[0].set_title('AND — 선형 분리 가능 ✓\n(하나의 직선으로 분리됨)', fontsize=11)
axes[0].set_xlim(-0.5, 1.5); axes[0].set_ylim(-0.5, 1.5)
axes[0].legend()

# XOR - 선형 분리 불가능
for xi, yi in zip(X_xor, y_xor):
    axes[1].scatter(xi[0], xi[1], c='tomato' if yi == 1 else 'steelblue',
                    s=300, zorder=5, marker='^' if yi == 1 else 's', edgecolors='black', lw=1.5)
    axes[1].annotate(f'({int(xi[0])},{int(xi[1])})={yi}', xy=(xi[0] + 0.07, xi[1] + 0.07), fontsize=11)
for angle in np.linspace(0, np.pi, 5):
    x_l = np.array([-0.5, 1.5])
    if np.cos(angle) != 0:
        y_l = 0.5 + np.tan(angle) * (x_l - 0.5)
        axes[1].plot(x_l, y_l, 'gray', lw=1, alpha=0.4, linestyle='--')
axes[1].set_title('XOR — 선형 분리 불가 ✗\n(어떤 직선으로도 분리 불가)', fontsize=11)
axes[1].set_xlim(-0.5, 1.5); axes[1].set_ylim(-0.5, 1.5)

plt.suptitle('퍼셉트론의 한계: XOR은 단일 직선으로 분리 불가\n→ 다층 퍼셉트론(MLP)이 필요!', fontsize=12)
plt.tight_layout()
plt.show()

---
## Part 2. 활성화 함수 (Activation Functions)

### 2-1. 왜 활성화 함수가 필요한가?

활성화 함수가 없다면 아무리 깊은 신경망도 **선형 변환**에 불과합니다.

$$W_3(W_2(W_1 x)) = W_{\text{합성}} x \quad (\text{선형})$$

비선형 활성화 함수 $f$를 추가하면 복잡한 함수를 근사할 수 있습니다.

### 2-2. 주요 활성화 함수 비교

| 함수 | 식 | 출력 범위 | 특징 |
|---|---|---|---|
| **Sigmoid** | $\frac{1}{1+e^{-z}}$ | (0, 1) | 확률 해석 용이, 기울기 소실 |
| **Tanh** | $\tanh(z)$ | (-1, 1) | 0 중심, 기울기 소실 |
| **ReLU** | $\max(0, z)$ | $[0, \infty)$ | 빠름, 기울기 소실 없음 |
| **Leaky ReLU** | $\max(0.01z, z)$ | $(-\infty, \infty)$ | 죽은 ReLU 개선 |

In [ ]:
# 활성화 함수 시각화
z = np.linspace(-5, 5, 300)

sigmoid   = 1 / (1 + np.exp(-z))
tanh_val  = np.tanh(z)
relu_val  = np.maximum(0, z)
lrelu_val = np.where(z > 0, z, 0.01 * z)

d_sigmoid  = sigmoid * (1 - sigmoid)
d_tanh     = 1 - tanh_val ** 2
d_relu     = np.where(z > 0, 1.0, 0.0)
d_lrelu    = np.where(z > 0, 1.0, 0.01)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

funcs = [
    ('Sigmoid',         sigmoid,   d_sigmoid,  'steelblue'),
    ('Tanh',            tanh_val,  d_tanh,     'tomato'),
    ('ReLU',            relu_val,  d_relu,     'seagreen'),
    ('Leaky ReLU (α=0.01)', lrelu_val, d_lrelu, 'darkorange'),
]

for col, (name, f_val, df_val, color) in enumerate(funcs):
    axes[0, col].plot(z, f_val, lw=2.5, color=color)
    axes[0, col].axhline(0, color='k', lw=0.8, linestyle=':')
    axes[0, col].axvline(0, color='k', lw=0.8, linestyle=':')
    axes[0, col].set_title(name, fontsize=11)
    axes[0, col].set_xlabel('z')
    axes[0, col].set_ylabel('f(z)' if col == 0 else '')

    axes[1, col].plot(z, df_val, lw=2.5, color=color, linestyle='--')
    axes[1, col].axhline(0, color='k', lw=0.8, linestyle=':')
    axes[1, col].axvline(0, color='k', lw=0.8, linestyle=':')
    axes[1, col].set_title(f"{name} 미분", fontsize=11)
    axes[1, col].set_xlabel('z')
    axes[1, col].set_ylabel("f'(z)" if col == 0 else '')
    axes[1, col].set_ylim(-0.1, 1.2)

plt.suptitle('활성화 함수와 미분 비교\n(미분이 0에 가까우면 기울기 소실 발생)', fontsize=12, y=1.01)
plt.tight_layout()
plt.show()

print('=== 최대 기울기 비교 (z=0에서) ===')
print(f'  Sigmoid 최대 기울기: {d_sigmoid.max():.4f}')
print(f'  Tanh    최대 기울기: {d_tanh.max():.4f}')
print(f'  ReLU    최대 기울기: {d_relu.max():.4f}  (z>0 구간)')
print()
print('→ Sigmoid/Tanh: 포화 영역에서 기울기 ≈ 0  → 기울기 소실(Vanishing Gradient)!')
print('→ ReLU: z>0에서 기울기 = 1  → 깊은 신경망에 적합')

In [ ]:
# 기울기 소실(Vanishing Gradient) 시뮬레이션
# 여러 층을 통과할 때 기울기가 어떻게 곱해지는지 확인

n_layers = 10

def simulate_gradient(activation, z_init, n_layers):
    grad = 1.0
    history = [1.0]
    z = z_init
    for _ in range(n_layers):
        if activation == 'sigmoid':
            s = 1 / (1 + np.exp(-z))
            grad *= s * (1 - s)
        elif activation == 'tanh':
            t = np.tanh(z)
            grad *= (1 - t ** 2)
        elif activation == 'relu':
            grad *= 1.0  # z>0 가정
        history.append(grad)
    return history

fig, axes = plt.subplots(1, 2, figsize=(13, 4))
layers = list(range(n_layers + 1))

# 포화 영역 (z=2.0)
for act, color, label in [
    ('sigmoid', 'steelblue', 'Sigmoid'),
    ('tanh',    'tomato',    'Tanh'),
    ('relu',    'seagreen',  'ReLU'),
]:
    grads = simulate_gradient(act, 2.0, n_layers)
    axes[0].semilogy(layers, grads, 'o-', color=color, lw=2, label=label)

axes[0].set_title('층별 누적 기울기 (포화 영역, z=2.0, 로그 스케일)')
axes[0].set_xlabel('층 수')
axes[0].set_ylabel('누적 기울기 (로그)')
axes[0].legend()

# 활성 영역 (z=0.5)
for act, color, label in [
    ('sigmoid', 'steelblue', 'Sigmoid'),
    ('tanh',    'tomato',    'Tanh'),
    ('relu',    'seagreen',  'ReLU'),
]:
    grads = simulate_gradient(act, 0.5, n_layers)
    axes[1].plot(layers, grads, 'o-', color=color, lw=2, label=label)

axes[1].set_title('층별 누적 기울기 (활성 영역, z=0.5)')
axes[1].set_xlabel('층 수')
axes[1].set_ylabel('누적 기울기')
axes[1].legend()

plt.suptitle('기울기 소실(Vanishing Gradient): 층이 깊어질수록 기울기가 사라진다', fontsize=12)
plt.tight_layout()
plt.show()

g_sig = simulate_gradient('sigmoid', 2.0, 10)
g_relu = simulate_gradient('relu', 2.0, 10)
print(f'Sigmoid 10층 누적 기울기 (포화): {g_sig[-1]:.2e}')
print(f'ReLU    10층 누적 기울기:        {g_relu[-1]:.2e}')
print('\n→ ReLU가 깊은 신경망 학습에 유리한 이유!')

---
## Part 3. 다층 퍼셉트론(MLP) — 순전파

### 3-1. MLP 아키텍처

```
입력층          은닉층 1        은닉층 2        출력층
  x₁ ──┐
  x₂ ──┼──→ [h₁¹] ──→ [h₁²] ──→ [ŷ]
  ...  └── [h₂¹] ──→ [h₂²]
              ...        ...
```

각 층의 계산:
$$z^{(l)} = W^{(l)} a^{(l-1)} + b^{(l)}, \quad a^{(l)} = f\left(z^{(l)}\right)$$

### 3-2. 순전파 직접 계산

2개 입력 → 4개 은닉 → 4개 은닉 → 1개 출력 구조를 NumPy로 직접 구현합니다.

In [ ]:
# MLP 순전파 직접 계산 (NumPy)
# 아키텍처: 2 → 4 → 4 → 1
np.random.seed(42)

W1 = np.random.randn(4, 2) * 0.5   # 은닉층 1 가중치
b1 = np.zeros(4)
W2 = np.random.randn(4, 4) * 0.5   # 은닉층 2 가중치
b2 = np.zeros(4)
W3 = np.random.randn(1, 4) * 0.5   # 출력층 가중치
b3 = np.zeros(1)


def relu(z):
    return np.maximum(0, z)


def sigmoid_np(z):
    return 1 / (1 + np.exp(-z))


def forward_pass(x):
    """2→4→4→1 MLP 순전파"""
    z1 = W1 @ x + b1        # 은닉층 1 가중합
    a1 = relu(z1)            # 은닉층 1 출력

    z2 = W2 @ a1 + b2       # 은닉층 2 가중합
    a2 = relu(z2)            # 은닉층 2 출력

    z3 = W3 @ a2 + b3       # 출력층 가중합
    a3 = sigmoid_np(z3)      # 출력층 (이진 분류)
    return a1, a2, a3, z1, z2, z3


x_sample = np.array([1.5, -0.5])
a1, a2, a3, z1, z2, z3 = forward_pass(x_sample)

print('=== MLP 순전파 (2 → 4 → 4 → 1) ===')
print(f'\n입력: {x_sample}')
print(f'\n[은닉층 1]')
print(f'  z1 = W1 @ x + b1 = {z1.round(4)}')
print(f'  a1 = ReLU(z1)     = {a1.round(4)}')
print(f'\n[은닉층 2]')
print(f'  z2 = W2 @ a1 + b2 = {z2.round(4)}')
print(f'  a2 = ReLU(z2)      = {a2.round(4)}')
print(f'\n[출력층]')
print(f'  z3 = W3 @ a2 + b3 = {z3.round(4)}')
print(f'  a3 = σ(z3)         = {a3.round(4)}')
print(f'\n예측 클래스: {int(a3[0] >= 0.5)}  (확률: {a3[0]:.4f})')

### 3-3. MLP로 XOR 해결

퍼셉트론이 풀 수 없었던 XOR 문제를 sklearn의 MLP로 해결합니다.

**핵심 아이디어:** 은닉층이 입력을 비선형 변환하여 선형 분리 가능한 공간으로 매핑합니다.

In [ ]:
from sklearn.neural_network import MLPClassifier

# XOR 데이터
X_xor_t = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=float)
y_xor_t = np.array([0, 1, 1, 0])

# 퍼셉트론 (한계 확인)
pct_xor2 = Perceptron(n_features=2, lr=0.1, n_epochs=100)
pct_xor2.fit(X_xor_t, y_xor_t)
acc_pct = accuracy_score(y_xor_t, pct_xor2.predict(X_xor_t))

# MLP (해결)
mlp_xor = MLPClassifier(hidden_layer_sizes=(4, 4), activation='relu',
                         max_iter=10000, random_state=42)
mlp_xor.fit(X_xor_t, y_xor_t)
acc_mlp = accuracy_score(y_xor_t, mlp_xor.predict(X_xor_t))

print('=== XOR 해결 비교 ===')
print(f'퍼셉트론 정확도: {acc_pct:.2f}  (선형 분리 불가)')
print(f'MLP       정확도: {acc_mlp:.2f}  (비선형 변환으로 해결!)')
print()
for xi, yi in zip(X_xor_t, y_xor_t):
    y_hat_mlp = mlp_xor.predict(xi.reshape(1, -1))[0]
    status = '✓' if yi == y_hat_mlp else '✗'
    print(f'  MLP: 입력 {xi.astype(int)} → 정답: {yi}, 예측: {y_hat_mlp} {status}')

# 결정 경계 비교 시각화
xx, yy = np.meshgrid(np.linspace(-0.5, 1.5, 200), np.linspace(-0.5, 1.5, 200))
grid = np.c_[xx.ravel(), yy.ravel()]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
for ax, model, title in [
    (axes[0], pct_xor2, f'퍼셉트론 (Acc={acc_pct:.2f})'),
    (axes[1], mlp_xor,  f'MLP  4→4 (Acc={acc_mlp:.2f})'),
]:
    Z = model.predict(grid).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdBu')
    ax.contour(xx, yy, Z, colors='k', linewidths=1.5)
    for xi, yi in zip(X_xor_t, y_xor_t):
        ax.scatter(xi[0], xi[1], c='tomato' if yi == 1 else 'steelblue',
                   s=300, zorder=5, marker='^' if yi == 1 else 's',
                   edgecolors='black', lw=1.5)
        ax.annotate(f'({int(xi[0])},{int(xi[1])})={yi}',
                    xy=(xi[0] + 0.07, xi[1] + 0.07), fontsize=12)
    ax.set_title(title, fontsize=12)
    ax.set_xlabel('x₁'); ax.set_ylabel('x₂')
    ax.set_xlim(-0.5, 1.5); ax.set_ylim(-0.5, 1.5)

plt.suptitle('XOR 문제: 퍼셉트론 vs MLP', fontsize=13)
plt.tight_layout()
plt.show()

---
## Part 4. PyTorch 기초 — Tensor & Autograd

### 4-1. 텐서(Tensor)

PyTorch의 핵심 자료구조입니다. NumPy 배열과 유사하지만 **GPU 가속** 및 **자동 미분(Autograd)** 을 지원합니다.

| 작업 | NumPy | PyTorch |
|---|---|---|
| 배열 생성 | `np.array(...)` | `torch.tensor(...)` |
| 난수 | `np.random.randn(...)` | `torch.randn(...)` |
| 차원 변경 | `.reshape(...)` | `.view(...)` / `.reshape(...)` |
| 행렬 곱 | `@` | `@` / `torch.matmul` |
| NumPy 변환 | — | `.numpy()` |

### 4-2. Autograd — 자동 미분

`requires_grad=True`를 설정하면 PyTorch가 계산 그래프를 자동으로 구성합니다.  
`.backward()`를 호출하면 체인 룰(Chain Rule)로 모든 파라미터의 기울기를 계산합니다.

In [ ]:
# PyTorch 텐서 기본 연산
print('=== PyTorch 텐서 기본 연산 ===\n')

a = torch.tensor([1.0, 2.0, 3.0])
b = torch.tensor([4.0, 5.0, 6.0])
print(f'a = {a}')
print(f'b = {b}')
print(f'a + b  = {a + b}')
print(f'a * b  = {a * b}')
print(f'a · b  = {torch.dot(a, b).item()}')

A = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
B = torch.tensor([[5.0, 6.0], [7.0, 8.0]])
print(f'\nA @ B =\n{A @ B}')

# NumPy 호환성
np_arr = np.array([1.0, 2.0, 3.0])
t_from_np = torch.from_numpy(np_arr)
print(f'\nNumPy → Tensor: {t_from_np}')
print(f'Tensor → NumPy: {t_from_np.numpy()}')

# 텐서 형태 변환
x = torch.randn(3, 4)
print(f'\nx.shape          = {x.shape}')
print(f'x.view(2,6).shape = {x.view(2, 6).shape}')
print(f'x.reshape(12).shape = {x.reshape(12).shape}')
print(f'x.T.shape        = {x.T.shape}')

In [ ]:
# Autograd — 자동 미분
print('=== Autograd: 기울기 자동 계산 ===\n')

# z = x² + 3y + 5 의 기울기 계산
x = torch.tensor(2.0, requires_grad=True)   # ∂z/∂x = 2x = 4
y = torch.tensor(3.0, requires_grad=True)   # ∂z/∂y = 3

z = x ** 2 + 3 * y + 5

print(f'x = {x.item()}, y = {y.item()}')
print(f'z = x² + 3y + 5 = {z.item()}')

z.backward()  # 역전파 실행

print(f'\n∂z/∂x = 2x = {x.grad.item():.1f}  (이론: {2 * 2.0:.1f})')
print(f'∂z/∂y = 3  = {y.grad.item():.1f}  (이론: 3.0)')

print('\n=== 신경망 파라미터 기울기 계산 ===')
# 단층 신경망: y_pred = σ(w·x + b)
w = torch.tensor(0.5, requires_grad=True)
b_param = torch.tensor(0.1, requires_grad=True)
x_in = torch.tensor(2.0)
y_true = torch.tensor(1.0)

z_net = w * x_in + b_param
y_pred = torch.sigmoid(z_net)
loss = (y_pred - y_true) ** 2   # MSE 손실

print(f'w={w.item():.2f}, b={b_param.item():.2f}, x={x_in.item():.1f}')
print(f'z = w·x + b = {z_net.item():.4f}')
print(f'ŷ = σ(z)   = {y_pred.item():.4f}')
print(f'loss = (ŷ - 1)² = {loss.item():.4f}')

loss.backward()
print(f'\n∂loss/∂w = {w.grad.item():.6f}')
print(f'∂loss/∂b = {b_param.grad.item():.6f}')
print('\n→ PyTorch가 체인 룰을 자동으로 적용!')

In [ ]:
# nn.Module로 신경망 구성
print('=== nn.Module 기초 ===')

# 방법 1: nn.Sequential (간단한 경우)
model_seq = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 4),
    nn.ReLU(),
    nn.Linear(4, 1),
    nn.Sigmoid()
)
print('nn.Sequential:')
print(model_seq)

# 파라미터 개수 확인
total_params = sum(p.numel() for p in model_seq.parameters())
print(f'\n총 파라미터 수: {total_params:,}')

# 순전파 테스트
x_test = torch.randn(5, 4)  # 배치 크기 5, 특성 4
y_test = model_seq(x_test)
print(f'\n입력 shape: {x_test.shape}')
print(f'출력 shape: {y_test.shape}')
print(f'출력값 (예측 확률): {y_test.detach().numpy().flatten().round(4)}')

---
## Part 5. PyTorch로 MLP 학습

### 5-1. PyTorch 학습 파이프라인

```python
for epoch in range(n_epochs):
    model.train()                     # 학습 모드
    for X_batch, y_batch in loader:
        optimizer.zero_grad()         # 기울기 초기화
        y_pred = model(X_batch)       # 순전파
        loss = criterion(y_pred, y_batch)  # 손실 계산
        loss.backward()               # 역전파 (기울기 계산)
        optimizer.step()              # 파라미터 업데이트
```

### 5-2. 데이터 준비 — Breast Cancer Wisconsin

In [ ]:
# 데이터 준비
cancer = load_breast_cancer(as_frame=True)
X_bc = cancer.data.values.astype(np.float32)
y_bc = cancer.target.values.astype(np.float32)

X_tr, X_te, y_tr, y_te = train_test_split(
    X_bc, y_bc, test_size=0.2, random_state=42, stratify=y_bc
)
scaler = StandardScaler()
X_tr_s = scaler.fit_transform(X_tr).astype(np.float32)
X_te_s = scaler.transform(X_te).astype(np.float32)

# PyTorch 텐서 변환
X_tr_t = torch.tensor(X_tr_s)
y_tr_t = torch.tensor(y_tr).unsqueeze(1)
X_te_t = torch.tensor(X_te_s)
y_te_t = torch.tensor(y_te).unsqueeze(1)

# DataLoader (미니배치)
train_ds = TensorDataset(X_tr_t, y_tr_t)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

print('Breast Cancer Wisconsin 데이터셋')
print(f'  특성 수   : {X_bc.shape[1]}')
print(f'  Train     : {len(X_tr)}개  (양성={int(y_tr.sum())}, 악성={len(y_tr)-int(y_tr.sum())})')
print(f'  Test      : {len(X_te)}개  (양성={int(y_te.sum())}, 악성={len(y_te)-int(y_te.sum())})')
print(f'\n텐서 형태: X_tr_t={X_tr_t.shape}, y_tr_t={y_tr_t.shape}')
print(f'배치 크기: 32  →  {len(train_loader)}개 배치/에포크')

In [ ]:
# MLP 모델 정의
class MLP(nn.Module):
    """다층 퍼셉트론 — PyTorch nn.Module 기반"""

    def __init__(self, input_dim, hidden_dims, output_dim=1,
                 activation='relu', dropout=0.0):
        super().__init__()

        act_map = {'relu': nn.ReLU(), 'sigmoid': nn.Sigmoid(), 'tanh': nn.Tanh()}

        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(act_map.get(activation, nn.ReLU()))
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


# 모델 구조 확인
model_demo = MLP(input_dim=30, hidden_dims=[64, 32], output_dim=1,
                 activation='relu', dropout=0.2)
print('=== MLP 모델 구조 (30 → 64 → 32 → 1) ===')
print(model_demo)
total_params = sum(p.numel() for p in model_demo.parameters())
print(f'\n총 파라미터 수: {total_params:,}')

In [ ]:
# 학습 함수
def train_model(model, train_loader, X_val, y_val,
                n_epochs=100, lr=0.001, optimizer_name='adam'):
    criterion = nn.BCELoss()

    if optimizer_name == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=lr)

    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(n_epochs):
        # 학습 단계
        model.train()
        epoch_loss, correct, total = 0.0, 0, 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * len(X_batch)
            correct += ((y_pred >= 0.5).float() == y_batch).sum().item()
            total += len(y_batch)

        train_losses.append(epoch_loss / total)
        train_accs.append(correct / total)

        # 검증 단계
        model.eval()
        with torch.no_grad():
            y_val_pred = model(X_val)
            val_loss = criterion(y_val_pred, y_val).item()
            val_acc = ((y_val_pred >= 0.5).float() == y_val).float().mean().item()
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        if (epoch + 1) % 25 == 0:
            print(f'Epoch {epoch+1:3d}/{n_epochs} | '
                  f'Train Loss: {train_losses[-1]:.4f} | '
                  f'Val Loss: {val_loss:.4f} | '
                  f'Val Acc: {val_acc:.4f}')

    return train_losses, val_losses, train_accs, val_accs


# MLP 학습 실행
torch.manual_seed(42)
model = MLP(input_dim=30, hidden_dims=[64, 32], output_dim=1,
            activation='relu', dropout=0.2)
print('=== MLP (30→64→32→1) 학습 시작 ===')
train_losses, val_losses, train_accs, val_accs = train_model(
    model, train_loader, X_te_t, y_te_t, n_epochs=100, lr=0.001
)

In [ ]:
# 학습 곡선 시각화
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
epochs = range(1, len(train_losses) + 1)

axes[0].plot(epochs, train_losses, 'b-', lw=2, label='Train Loss')
axes[0].plot(epochs, val_losses,   'r-', lw=2, label='Val Loss')
axes[0].set_title('Loss 곡선')
axes[0].set_xlabel('에포크')
axes[0].set_ylabel('BCE Loss')
axes[0].legend()

axes[1].plot(epochs, train_accs, 'b-', lw=2, label='Train Accuracy')
axes[1].plot(epochs, val_accs,   'r-', lw=2, label='Val Accuracy')
axes[1].set_title('Accuracy 곡선')
axes[1].set_xlabel('에포크')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].set_ylim(0.7, 1.02)

plt.suptitle('MLP 학습 곡선 (Breast Cancer)', fontsize=13)
plt.tight_layout()
plt.show()

# 최종 성능
model.eval()
with torch.no_grad():
    y_pred_final = model(X_te_t).numpy()
y_labels = (y_pred_final >= 0.5).astype(int).flatten()
test_acc = accuracy_score(y_te, y_labels)
test_auc = roc_auc_score(y_te, y_pred_final.flatten())
print(f'최종 Test Accuracy : {test_acc:.4f}')
print(f'최종 Test ROC-AUC  : {test_auc:.4f}')

In [ ]:
# 옵티마이저 비교: Adam vs SGD vs RMSprop
print('=== 옵티마이저 비교 (각 100 에포크) ===\n')

optimizer_configs = [
    ('Adam',    'adam',    0.001),
    ('SGD',     'sgd',     0.01),
    ('RMSprop', 'rmsprop', 0.001),
]

fig, axes = plt.subplots(1, 2, figsize=(13, 4))
colors_opt = ['steelblue', 'tomato', 'seagreen']
epochs_range = range(1, 101)

for (name, opt_name, lr), color in zip(optimizer_configs, colors_opt):
    torch.manual_seed(42)
    m = MLP(input_dim=30, hidden_dims=[64, 32], output_dim=1, activation='relu')
    t_l, v_l, t_a, v_a = train_model(m, train_loader, X_te_t, y_te_t,
                                      n_epochs=100, lr=lr, optimizer_name=opt_name)
    axes[0].plot(epochs_range, v_l, color=color, lw=2, label=f'{name} (lr={lr})')
    axes[1].plot(epochs_range, v_a, color=color, lw=2,
                 label=f'{name} (최종: {v_a[-1]:.4f})')
    print(f'{name:>8}: 최종 Val Acc={v_a[-1]:.4f}  Val Loss={v_l[-1]:.4f}')

axes[0].set_title('옵티마이저별 Val Loss')
axes[0].set_xlabel('에포크'); axes[0].set_ylabel('BCE Loss')
axes[0].legend()

axes[1].set_title('옵티마이저별 Val Accuracy')
axes[1].set_xlabel('에포크'); axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].set_ylim(0.85, 1.02)

plt.suptitle('옵티마이저 비교', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# sklearn 모델과 성능 비교
sklearn_models = {
    'Logistic Regression':   LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest':         RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting':     GradientBoostingClassifier(n_estimators=100, random_state=42),
}

results_cmp = {}
for name, m in sklearn_models.items():
    m.fit(X_tr_s, y_tr)
    y_pred = m.predict(X_te_s)
    y_prob = m.predict_proba(X_te_s)[:, 1]
    results_cmp[name] = (accuracy_score(y_te, y_pred), roc_auc_score(y_te, y_prob))

# MLP (PyTorch) 결과 추가
results_cmp['MLP (PyTorch)'] = (test_acc, test_auc)

# 출력
print(f'{"모델":<28} {"Accuracy":>10} {"ROC-AUC":>10}')
print('-' * 52)
for name, (acc, auc) in sorted(results_cmp.items(), key=lambda x: x[1][1]):
    best_mark = ' ★' if auc == max(v[1] for v in results_cmp.values()) else ''
    print(f'{name:<28} {acc:>10.4f} {auc:>10.4f}{best_mark}')

# 시각화
sorted_items = sorted(results_cmp.items(), key=lambda x: x[1][1])
names_cmp = [n for n, _ in sorted_items]
accs_cmp  = [v[0] for _, v in sorted_items]
aucs_cmp  = [v[1] for _, v in sorted_items]
bar_colors = ['#aed6f1' if 'MLP' not in n else 'steelblue' for n in names_cmp]

fig, axes = plt.subplots(1, 2, figsize=(13, 4))
for ax, vals, title in [(axes[0], accs_cmp, 'Test Accuracy'),
                         (axes[1], aucs_cmp, 'Test ROC-AUC')]:
    bars = ax.barh(names_cmp, vals, color=bar_colors, edgecolor='k', alpha=0.85)
    for bar, v in zip(bars, vals):
        ax.text(v + 0.001, bar.get_y() + bar.get_height() / 2,
                f'{v:.4f}', va='center', fontsize=9)
    ax.set_title(f'모델별 {title}')
    ax.set_xlabel(title)
    ax.set_xlim(0.9, 1.02)

plt.suptitle('MLP vs sklearn 모델 성능 비교 (Breast Cancer)', fontsize=12)
plt.tight_layout()
plt.show()

---
## Exercise

### Exercise 1. PyTorch로 OR 게이트 학습

`nn.Sequential`을 사용하여 OR 게이트를 학습하는 단층 퍼셉트론을 구현하세요.

| 입력 (x1, x2) | OR 출력 |
|---|---|
| (0, 0) | 0 |
| (0, 1) | 1 |
| (1, 0) | 1 |
| (1, 1) | 1 |

**요구사항:**
- `nn.Sequential(nn.Linear(2, 1), nn.Sigmoid())` 구성
- BCE 손실 함수 / SGD 옵티마이저 (lr=0.5)
- 200 에포크 학습
- 에포크별 손실 시각화
- 4개 입력 모두에 대한 예측 결과 출력

In [ ]:
# Exercise 1: OR 게이트 학습
X_or = torch.tensor([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
y_or = torch.tensor([[0.], [1.], [1.], [1.]])

# Your code here: 모델 정의 (nn.Sequential)
model_or = None

# Your code here: 손실 함수, 옵티마이저 정의

# Your code here: 200 에포크 학습

# Your code here: 에포크별 손실 시각화

# Your code here: 결과 출력 (4개 입력에 대한 예측)

### Exercise 2. 은닉층 크기 효과 분석

Breast Cancer 데이터에서 아래 4가지 아키텍처를 비교하세요.

| 모델 이름 | hidden_dims |
|---|---|
| Shallow | `[16]` |
| Medium | `[64, 32]` |
| Deep | `[128, 64, 32]` |
| Wide | `[256]` |

**요구사항:**
- 각 모델: `n_epochs=100`, `lr=0.001`, Adam 옵티마이저
- Val Loss 및 Val Accuracy 학습 곡선 시각화 (4개 모델 비교)
- 최종 Val Accuracy 비교 출력

In [ ]:
# Exercise 2: 아키텍처 비교
architectures = {
    'Shallow [16]':        [16],
    'Medium [64, 32]':     [64, 32],
    'Deep [128, 64, 32]':  [128, 64, 32],
    'Wide [256]':          [256],
}

arch_results = {}

for name, hidden_dims in architectures.items():
    # Your code here: 각 아키텍처 학습 및 결과 저장
    pass

# Your code here: Val Loss & Val Accuracy 학습 곡선 시각화

# Your code here: 최종 Val Accuracy 비교 출력

### Exercise 3. (도전) 학습률 스케줄링

Adam 옵티마이저에 학습률 스케줄러를 적용하여 세 가지 전략을 비교하세요.

| 전략 | 설명 |
|---|---|
| **고정 lr** | `lr=0.001` 고정 |
| **StepLR** | `step_size=30, gamma=0.5` (30 에포크마다 절반) |
| **CosineAnnealingLR** | `T_max=100` (코사인 형태로 감소) |

**요구사항:**
- 은닉층: `[64, 32]`, `n_epochs=100`, 초기 `lr=0.001`
- Val Accuracy 비교 곡선 시각화
- 학습률 변화 곡선 시각화

**힌트:**
```python
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
# 에포크 끝마다:
scheduler.step()
# 현재 학습률:
current_lr = optimizer.param_groups[0]['lr']
```

In [ ]:
# Exercise 3: 학습률 스케줄링
def train_with_scheduler(hidden_dims, n_epochs, lr, scheduler_type='none'):
    """
    scheduler_type: 'none', 'step', 'cosine'
    Returns: val_accs, lr_history
    """
    torch.manual_seed(42)
    model_sch = MLP(input_dim=30, hidden_dims=hidden_dims, output_dim=1, activation='relu')
    criterion_sch = nn.BCELoss()
    optimizer_sch = optim.Adam(model_sch.parameters(), lr=lr)

    # Your code here: 스케줄러 생성 (scheduler_type에 따라 다르게)

    val_accs = []
    lr_history = []

    for epoch in range(n_epochs):
        # Your code here: 학습 루프

        # Your code here: 검증 및 val_acc 기록

        # Your code here: 학습률 기록 및 scheduler.step() 호출
        pass

    return val_accs, lr_history


# Your code here: 세 가지 전략으로 학습 실행

# Your code here: Val Accuracy 비교 및 학습률 곡선 시각화

---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **퍼셉트론** | 입력 × 가중치 + 편향 → 활성화 함수 → 출력 |
| **퍼셉트론 한계** | 선형 분리 불가능 문제(XOR) 해결 불가 |
| **MLP** | 은닉층 추가 → 비선형 문제 해결 가능 |
| **Sigmoid** | (0,1) 범위, 이진 출력, 포화 시 기울기 소실 |
| **ReLU** | z>0이면 기울기=1, 기울기 소실 방지, 현재 가장 많이 사용 |
| **순전파** | $z = Wx + b \to a = f(z)$, 층별 반복 |
| **역전파** | 체인 룰로 기울기 계산, 출력→입력 방향 |
| **PyTorch Tensor** | NumPy + GPU + Autograd |
| **Autograd** | `requires_grad=True` → `.backward()` → `.grad` |
| **nn.Module** | 모델 정의, `forward()` 구현 |
| **학습 루프** | `zero_grad → forward → loss → backward → step` |
| **Adam** | 모멘텀 + 적응형 학습률, 가장 널리 사용 |

---

**다음 강의 (Week 11):** CNN — 합성곱 신경망, 풀링, 이미지 분류